In [32]:
import re
import string

import contractions
import gensim
import nltk
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [35]:
# https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
# import SentimentIntensityAnalyzer class 
# from vaderSentiment.vaderSentiment module. 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
  
# function to print sentiments 
# of the sentence. 
def sentiment_scores(sentence): 
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
    
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
      
    print("Overall sentiment dictionary is : ", sentiment_dict) 
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
    
    print("Sentence Overall Rated As", end = " ") 
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        print("Positive") 
    elif sentiment_dict['compound'] <= - 0.05 : 
        print("Negative") 
    else : 
        print("Neutral") 

In [36]:
# word2vec
path = get_tmpfile("word2vec.model")
model = Word2Vec(common_texts,
                 size=256,
                 window=5,
                 min_count=1,
                 workers=16,
                 sg=1)
model.save("word2vec.model")
# # conti training
# model = Word2Vec.load("word2vec.model")
# model.train([["hello", "world"]], total_examples=1, epochs=1)

In [43]:
ted_main = pd.read_csv('data/ted_main.csv')

In [44]:
ted_main.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [45]:
transcripts = pd.read_csv('data/transcripts.csv')

In [46]:
ted_merged = pd.merge(left=transcripts, right=ted_main, left_on='url', right_on='url')

In [47]:
ted_merged.head()

,transcript,url,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,views
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,47227110
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,3200520
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,1636292
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,1697550
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,12005869


In [49]:
transcript = ted_merged.transcript

In [50]:
def rm_num(input_str):
    return re.sub(r'\d+', '', input_str)

transcript = transcript.apply(contractions.fix) # remove contractions
transcript = transcript.str.lower() # lowercase
transcript = transcript.apply(rm_num) # remove numbers
specials = '()'
transcript = transcript.str.translate(str.maketrans('()',' '*len(specials))) # remove par around laughter
transcript = transcript.str.translate(str.maketrans('','',string.punctuation)) # remove punctuation
transscipt = transcript.str.strip()

In [51]:
# setiment analysis
transscipts.apply(sentiment_scores)

Overall sentiment dictionary is :  {'neg': 0.043, 'neu': 0.805, 'pos': 0.152, 'compound': 0.9999}
sentence was rated as  4.3 % Negative
sentence was rated as  80.5 % Neutral
sentence was rated as  15.2 % Positive
Sentence Overall Rated As Positive
Overall sentiment dictionary is :  {'neg': 0.038, 'neu': 0.82, 'pos': 0.142, 'compound': 0.9998}
sentence was rated as  3.8 % Negative
sentence was rated as  82.0 % Neutral
sentence was rated as  14.2 % Positive
Sentence Overall Rated As Positive
Overall sentiment dictionary is :  {'neg': 0.062, 'neu': 0.782, 'pos': 0.155, 'compound': 0.9999}
sentence was rated as  6.2 % Negative
sentence was rated as  78.2 % Neutral
sentence was rated as  15.5 % Positive
Sentence Overall Rated As Positive
Overall sentiment dictionary is :  {'neg': 0.073, 'neu': 0.774, 'pos': 0.153, 'compound': 0.9998}
sentence was rated as  7.3 % Negative
sentence was rated as  77.4 % Neutral
sentence was rated as  15.299999999999999 % Positive
Sentence Overall Rated As Posi

KeyboardInterrupt: 

In [18]:
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# transcripts[0]

In [298]:
# model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model = KeyedVectors.load_word2vec_format('~/weight/GoogleNews-vectors-negative300.bin', binary=True)
# model.wv['computer']

In [19]:
# ! nltk.download('punkt')
stemmer = PorterStemmer()
vec_embeddings = {}
for i, transcript in enumerate(transcripts):
    vec_embeddings[i] = [stemmer.stem(word) for word in word_tokenize(transcript) if not word in stop_words]

In [20]:
vec_embeddings[0]

['good',
 'morn',
 'laughter',
 'great',
 'blown',
 'away',
 'whole',
 'thing',
 'fact',
 'leav',
 'laughter',
 'three',
 'theme',
 'run',
 'confer',
 'relev',
 'want',
 'talk',
 'one',
 'extraordinari',
 'evid',
 'human',
 'creativ',
 'present',
 'peopl',
 'varieti',
 'rang',
 'second',
 'put',
 'us',
 'place',
 'idea',
 'go',
 'happen',
 'term',
 'futur',
 'idea',
 'may',
 'play',
 'outi',
 'interest',
 'educ',
 'actual',
 'find',
 'everybodi',
 'interest',
 'educ',
 'find',
 'interest',
 'dinner',
 'parti',
 'say',
 'work',
 'educ',
 '—',
 'actual',
 'often',
 'dinner',
 'parti',
 'frankli',
 'laughter',
 'work',
 'educ',
 'ask',
 'laughter',
 'never',
 'ask',
 'back',
 'curious',
 'strang',
 'say',
 'somebodi',
 'know',
 'say',
 'say',
 'work',
 'educ',
 'see',
 'blood',
 'run',
 'face',
 'like',
 'oh',
 'god',
 'know',
 'laughter',
 'one',
 'night',
 'week',
 'laughter',
 'ask',
 'educ',
 'pin',
 'wall',
 'one',
 'thing',
 'goe',
 'deep',
 'peopl',
 'right',
 'like',
 'religion',
